In [3]:
# From the course: Bayesin Machine Learning in Python: A/B Testing
# https://deeplearningcourses.com/c/bayesian-machine-learning-in-python-ab-testing
# https://www.udemy.com/bayesian-machine-learning-in-python-ab-testing
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import numpy as np
from flask import Flask, jsonify, request
from scipy.stats import beta

# create an app
app = Flask(__name__)


# define bandits
# there's no "pull arm" here
# since that's technically now the user/client
class Bandit:
  def __init__(self, name):
    self.clks = 0
    self.views = 0
    self.name = name

  def sample(self):
    # Beta(1, 1) is the prior
    a = 1 + self.clks
    b = 1 + self.views - self.clks
    return np.random.beta(a, b)

  def add_click(self):
    self.clks += 1

  def add_view(self):
    self.views += 1

  # TODO - what else does the Bandit need to do?
  # print some helpful stats
    if self.views % 50 == 0:
      print("%s: clks=%s, views=%s" % (self.name, self.clks, self.views))


# initialize bandits
banditA = Bandit('A')
banditB = Bandit('B')



@app.route('/get_ad')
def get_ad():
  if banditA.sample() > banditB.sample():
    ad = 'A'
    banditA.add_view()
  else:
    ad = 'B'
    banditB.add_view()
  return jsonify({'advertisement_id': 'A'})


@app.route('/click_ad', methods=['POST'])
def click_ad():
  result = 'OK'
  if request.form['advertisement_id'] == 'A':
    banditA.add_click()
  elif request.form['advertisement_id'] == 'B':
    banditB.add_click()
  else:
    result = 'Invalid Input.'

  # nothing to return really
  return jsonify({'result': result})


if __name__ == '__main__':
  app.run(host='127.0.0.1', port='8888')

UnsupportedOperation: not writable

In [2]:
# From the course: Bayesin Machine Learning in Python: A/B Testing
# https://deeplearningcourses.com/c/bayesian-machine-learning-in-python-ab-testing
# https://www.udemy.com/bayesian-machine-learning-in-python-ab-testing
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import requests
import numpy as np
import pandas as pd
from scipy import stats

# get data
df = pd.read_csv('advertisement_clicks.csv')
a = df[df['advertisement_id'] == 'A']
b = df[df['advertisement_id'] == 'B']
a = a['action'].values
b = b['action'].values

print("a.mean:", a.mean())
print("b.mean:", b.mean())


i = 0
j = 0
count = 0
while i < len(a) and j < len(b):
  # quit when there's no data left for either ad
  r = requests.get('http://localhost:8888/get_ad')
  # print(r.content)
  r = r.json()
  if r['advertisement_id'] == 'A':
    action = a[i]
    i += 1
  else:
    action = b[j]
    j += 1

  if action == 1:
    # only click the ad if our dataset determines that we should
    requests.post(
      'http://localhost:8888/click_ad',
      data={'advertisement_id': r['advertisement_id']}
    )

  # log some stats
  count += 1
  if count % 50 == 0:
    print("Seen %s ads, A: %s, B: %s" % (count, i, j))

a.mean: 0.304
b.mean: 0.372


JSONDecodeError: Expecting value: line 1 column 1 (char 0)